In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from datetime import date  # Make sure you import this at the top


In [2]:
base_url = 'https://a1buildtech.com/'
page = requests.get(base_url)
soup = BeautifulSoup(page.text,'html')
print(soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-KSDSQR2N');</script>
<!-- End Google Tag Manager -->
<meta content="4kXKTpc_wCA_vpe8g3FTfxAzHX1-JgbA9wlUArY88l0" name="google-site-verification"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<meta content="telephone=no" name="format-detection"/>
<title>A1 Buildtech – Real Estate Group</title>
<style>.houzez-library-modal-btn {margin-left: 5px;background: #35AAE1;vertical-align: top;font-size: 0 !important;}.houzez-library-modal-btn:before {content: '';width: 16px;height: 16px;bac

In [3]:
# list to store data
BHK_Type = []
Area_range =[]
Prices=[]
property_status = []
Area_unit=[]
RERA_number = []
property_type=[]
About_us = []
fig_link=[]
Amenities = []
highlights=[]
landing_url=[]
base_url=[]
done_by=[]
date_list=[]
Location=[]

In [4]:
# Properting Names
property_names = [h2.find('a').text for h2 in soup.find_all('h2', class_='item-title')]
property_names

['Puri 81 Business Hub',
 'Amolik Plaza',
 'Bptp Park Floor 2',
 'Puri 81 High Street',
 'Sarvome Shree Homes Phase 2',
 'Shiv Sai Ozone Park']

In [5]:
# Data Scrap flow

for property_name in property_names:
    url = f'http://a1buildtech.com/property/{property_name}/'
    req = requests.get(url)
    landing_url.append(url)

    if req.status_code == 200:
        soupall = BeautifulSoup(req.text, 'html.parser')

        # Title
        title = soupall.find('h1')
        if title:
            print(f"Title for {property_name}: {title.text.strip()}")

        # Property Type
        ptype = soupall.find('li', class_='prop_type')
        if ptype:
            full_text = ptype.text.strip()
            cleaned_text = full_text.replace("Property Type:", "").strip()
            print(f"Property_type: {cleaned_text}")
            property_type.append(cleaned_text)
        else:
            property_type.append("")

        # Property Status
        pstatus = soupall.find('li', class_='prop_status')
        if pstatus:
            full_text = pstatus.text.strip()
            cleaned_text = full_text.replace("Property Status:", "").strip()
            print(f"Property_Status: {cleaned_text}")
            property_status.append(cleaned_text)
        else:
            property_status.append("")

        # p_tags -> BHK
        p_tags = soupall.find_all('p')
        if len(p_tags) > 1:
            bhk_type = p_tags[1]
            bhk_text = bhk_type.get_text()
            configurations = re.findall(r'\d\s?BHK(?:\s?\+\s?\d)?', bhk_text)
            configurations = list(set(configurations))
            print(configurations)
            BHK_Type.append(", ".join(configurations))
        else:
            BHK_Type.append("")

        # Area Range
        if p_tags:
            p_text = " ".join(p.get_text(strip=True) for p in p_tags)
            area_range = re.findall(r'(\d{3,4})\s*sq\.?\s*ft', p_text, re.IGNORECASE)
            if area_range:
                if len(area_range) >= 2:
                    print(f"Area Range: {area_range[0]} sq.ft to {area_range[1]} sq.ft")
                    Area_range.append(f"{area_range[0]} to {area_range[1]}")
                else:
                    print(f"Single Area: {area_range[0]} sq.ft")
                    Area_range.append(f'{area_range[0]}')
            else:
                Area_range.append("")
        else:
            Area_range.append("")

        # Prices
        price = soupall.find('li', class_="item-price item-price-text price-single-listing-text")
        if price:
            print(f'Price :{price.text.strip()}')
            Prices.append(price.text.strip())
        else:
            Prices.append("")

        # RERA Number
        table = soupall.find('tbody')
        rera_found = False
        if table:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    heading = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)
                    if 'Rera No' in heading or 'RERA No' in heading:
                        print(f"RERA Number: {value}")
                        RERA_number.append(value)
                        rera_found = True
                        break
        if not rera_found:
            RERA_number.append("")

        # About Us
        table = soupall.find('table')
        if table:
            rows = table.find_all('tr')
            about_table_items = []
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    heading = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)
                    about_table_items.append(f'{heading}: {value}')
            about_table_sentence = ' | '.join(about_table_items)

            para = soupall.find_all('p')
            para_texts = [p.get_text(strip=True) for p in para]
            joined_text = ' '.join(para_texts)

            about_text = f'{joined_text} ({about_table_sentence})'
            print(f'About us: {about_text}')
            About_us.append(about_text)
        else:
            About_us.append("")

        # Image links
        figures = soupall.find_all('figure')
        image_link = ""
        for fig in figures:
            a_tag = fig.find('a')
            if a_tag and a_tag.has_attr('href'):
                print(a_tag['href'])
                image_link = a_tag['href']
                break
        fig_link.append(image_link)

        # Amenities
        table = soupall.find('tbody')
        amenities_found = False
        if table:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    heading = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)
                    if 'Amenities' in heading or 'amenities' in heading:
                        print(f"Amenities: {value}")
                        Amenities.append(value)
                        amenities_found = True
                        break
        if not amenities_found:
            Amenities.append("")

        # Highlights
        tables = soupall.find_all('table')
        if len(tables) > 1:
            table = tables[1]
            rows = table.find_all('tr')
            highlight_items = []
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    heading = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)
                    highlight_items.append(f'{heading}: {value}')
            final_sentence = ' | '.join(highlight_items)
            print(final_sentence)
            highlights.append(final_sentence)
        else:
            highlights.append("")

        # Location
        table = soupall.find('tbody')
        location_found = False
        if table:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    heading = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)
                    if 'Location' in heading:
                        print(f"Location: {value}")
                        Location.append(value)
                        location_found = True
                        break
        if not location_found:
            Location.append("")

        # Done by
        done_by.append('Akshay')

    else:
        print(f"Failed to fetch page for {property_name}, Status Code: {req.status_code}")


Title for Puri 81 Business Hub: Puri 81 Business Hub
Property_type: Commercial
Property_Status: Ready to Move
[]
Area Range: 485 sq.ft to 880 sq.ft
Price :₹ 27.67 Lac - ₹ 52.36 Lac
About us: Puri Constructions the builders par excellence in Faridabad with their genius style in construction of highly successful and luxurious commercial and residential spaces give you their new Gem, Puri 81 Business Hub. 81 Business Hub is a project of contemporary office spaces in Faridabad. Constructed in a high-rise tower in sector 81 Faridabad it has more than 270 offices in different sizes spread over a building of G + 13 stores.  The sizes of the offices vary from 485 Sq. Ft to 880 sq. Ft. There are four different sizes available. 485 sq. ft, 500 sq. Ft, 780 sq. ft and 880 sq. Ft. Puri 81 Business Hub enjoys the advantage of connectivity to the most expensive sectors in the whole city. The Sector 15, 16 and 17 and the Sector 21 are a just a 100-meter distance and a five-minute drive from the hub as

In [6]:

# After creating the DataFrame
df = pd.DataFrame({
    'Landing_URL': landing_url,
    'Location': Location,
    'property_name': property_names,
    'Property_Type': property_type,
    'BHK_Type': BHK_Type,
    'Area_Range': Area_range,
    'Property_Status': property_status,
    'Price': Prices,
    'RERA_Number': RERA_number,
    'About_Us': About_us,
    'Amenities': Amenities,
    'Image_Link': fig_link,
    'Highlights': highlights,
    'Done_By': done_by
})

# Save to Excel
df.to_excel("Assign.xlsx", index=False)

print("Excel file 'Assign.xlsx' created successfully!")


Excel file 'Assign.xlsx' created successfully!


In [9]:
df

,Landing_URL,Location,property_name,Property_Type,BHK_Type,Area_Range,Property_Status,Price,RERA_Number,About_Us,Amenities,Image_Link,Highlights,Done_By
0,http://a1buildtech.com/property/Puri 81 Busine...,Sector 81,Puri 81 Business Hub,Commercial,,485 to 880,Ready to Move,₹ 27.67 Lac - ₹ 52.36 Lac,,Puri Constructions the builders par excellence...,"Gated Community, 24×7 Security, Shopping Compl...",https://www.propertymaster.in/property/puri-81...,Bypass Road: 200 Mtr. | Delhi-Mathura Road: 2 ...,Akshay
1,http://a1buildtech.com/property/Amolik Plaza/,Sector 81 Faridabad,Amolik Plaza,Commercial,,,Under Construction,₹ 1.26 Cr. - ₹ 3.26 Cr.,,Ever since Faridabad was planned there was a v...,,https://www.propertymaster.in/property/amolik-...,Metro Station: 2 KM | Delhi-Mumbai Expressway:...,Akshay
2,http://a1buildtech.com/property/Bptp Park Floo...,"Sector 76, Faridabad",Bptp Park Floor 2,Commercial,"2 BHK, 3 BHK",1037 to 1189,Ready to Move,₹ 30 Lac - ₹ 48 Lac,,BPTP the most reputable real estate builders a...,"Gated Community, 24×7 Security, Central Green ...",https://www.propertymaster.in/property/bptp-pa...,Bptp Next Door Mall: 100 Mtr. | Bypass Road: 2 KM,Akshay
3,http://a1buildtech.com/property/Puri 81 High S...,Sector 81,Puri 81 High Street,Commercial,,160 to 825,Ready to Move,₹ 30 Lac - ₹ 90 Lac,,Puri Constructions is one of the best construc...,"Gated Community, 24×7 Security, Shopping Compl...",https://www.propertymaster.in/property/puri-81...,Hospital: 2 KM | Bypass Road: 200 Meter,Akshay
4,http://a1buildtech.com/property/Sarvome Shree ...,"Sector 45, Faridabad",Sarvome Shree Homes Phase 2,Residential,,765 to 765,Under Construction,₹ 26.33 Lac - ₹ 26.33 Lac,HRERA-PKL-FBD-171-2019,Lorem Ipsumis simply dummy text of the printin...,"Gated Community, 24×7 Security, Central Green ...",https://www.propertymaster.in/property/sarvome...,NH2: 800 Mtr. | Metro Station: 1 KM | South De...,Akshay
5,http://a1buildtech.com/property/Shiv Sai Ozone...,Sector 86 Faridabad,Shiv Sai Ozone Park,Residential,"2 BHK, 3 BHK + 1, 3 BHK",1150 to 1909,Ready to Move,₹ 58 Lac - ₹ 85 Lac,,Shiv Sai infrastructure is a construction comp...,"Gated Community, 24×7 Security, Club House wit...",https://www.propertymaster.in/property/shiv-sa...,Bypass Road: 1 KM | Shopping Mall: 0.5 KM,Akshay
